<a href="https://colab.research.google.com/github/hexeeyy/scoliosis/blob/main/scoliosis.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

**Import necessary libraries**

In [3]:
!pip install mediapipe

import numpy as np
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers
import cv2
import json
import os
import mediapipe as mp
import math
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from sklearn.metrics import confusion_matrix, classification_report
import seaborn as sns
from google.colab import drive
from tensorflow.keras.regularizers import l2
from sklearn.utils.class_weight import compute_class_weight

# Print versions
print("✅ NumPy version:", np.__version__)
print("✅ TensorFlow version:", tf.__version__)

✅ NumPy version: 1.26.4
✅ TensorFlow version: 2.18.0


In [4]:
# Mount Google Drive
drive.mount('/content/drive', force_remount=True)

Mounted at /content/drive


In [5]:
# Set dataset path
dataset_path = "/content/drive/MyDrive/final_na_talaga/dataset"

# Check if the dataset path exists
if os.path.exists(dataset_path):
    print("✅ Dataset path exists!")
    # List folders and files inside
    print("Contents:", os.listdir(dataset_path))
else:
    print("❌ Dataset path does NOT exist. Check the path again.")

✅ Dataset path exists!
Contents: ['data.yaml', 'resnet50_scoliosis_model.h5', 'severe', 'moderate', 'normal', 'mild']


In [6]:
# Initialize MediaPipe Pose (optional, not used here but kept for compatibility)
mp_pose = mp.solutions.pose
pose = mp_pose.Pose()

In [7]:
# Function to calculate Cobb angle from keypoints (simplified)
def calculate_cobb_angle(keypoints):
    keypoints = keypoints.reshape(-1, 2)  # Reshape to (17, 2)
    upper_spine = (keypoints[5] + keypoints[6]) / 2  # Midpoint of shoulders
    lower_spine = (keypoints[11] + keypoints[12]) / 2  # Midpoint of hips
    delta_y = upper_spine[1] - lower_spine[1]
    delta_x = upper_spine[0] - lower_spine[0]
    angle_radians = math.atan2(delta_y, delta_x)
    angle_degrees = math.degrees(angle_radians)
    return min(abs(angle_degrees), 90)  # Cap at 90°

# Load keypoints and calculate Cobb angles from JSON
def load_keypoints_and_angles_from_json(base_folder):
    keypoints_data = []
    cobb_angles = []
    labels = []
    class_folders = ['normal', 'mild', 'moderate', 'severe']
    class_mapping = {'normal': 0, 'mild': 1, 'moderate': 2, 'severe': 3}
    expected_length = 34  # 17 keypoints * (x, y)

    for class_name in class_folders:
        class_path = os.path.join(base_folder, class_name)
        for split in ['train', 'valid', 'test']:
            split_path = os.path.join(class_path, split)
            annotation_file = os.path.join(split_path, '_annotations.coco.json')
            if os.path.exists(annotation_file):
                with open(annotation_file, 'r') as f:
                    annotations = json.load(f)
                    for ann in annotations['annotations']:
                        keypoints = np.array(ann['keypoints']).reshape(-1, 3)[:, :2].flatten()
                        if len(keypoints) < expected_length:
                            keypoints = np.pad(keypoints, (0, expected_length - len(keypoints)), 'constant')
                        elif len(keypoints) > expected_length:
                            keypoints = keypoints[:expected_length]
                        keypoints_data.append(keypoints)
                        cobb_angles.append(calculate_cobb_angle(keypoints))
                        labels.append(class_mapping[class_name])

    return np.array(keypoints_data, dtype=np.float32), np.array(cobb_angles, dtype=np.float32), np.array(labels, dtype=np.int32)

# Data augmentation function
def augment_keypoints(keypoints, max_shift=0.05, max_scale=0.1, max_rotate=10):
    keypoints = keypoints.reshape(-1, 2)  # Reshape to (17, 2)
    shift_x = np.random.uniform(-max_shift, max_shift) * np.max(keypoints[:, 0])
    shift_y = np.random.uniform(-max_shift, max_shift) * np.max(keypoints[:, 1])
    keypoints[:, 0] += shift_x
    keypoints[:, 1] += shift_y
    scale = 1 + np.random.uniform(-max_scale, max_scale)
    keypoints *= scale
    angle = np.radians(np.random.uniform(-max_rotate, max_rotate))
    cos_val, sin_val = np.cos(angle), np.sin(angle)
    rotation_matrix = np.array([[cos_val, -sin_val], [sin_val, cos_val]])
    keypoints = keypoints.dot(rotation_matrix)
    return keypoints.flatten()

# Load dataset
X, cobb_angles, y = load_keypoints_and_angles_from_json(dataset_path)
X = X / np.max(X)  # Normalize keypoints
cobb_angles = cobb_angles / 90.0  # Normalize angles to [0, 1]

# Split dataset: 96% train, 3% validation, 1% test
X_train, X_temp, y_train, y_temp, cobb_train, cobb_temp = train_test_split(
    X, y, cobb_angles, test_size=0.04, random_state=42
)
X_val, X_test, y_val, y_test, cobb_val, cobb_test = train_test_split(
    X_temp, y_temp, cobb_temp, test_size=0.25, random_state=42
)

# Augment training data
X_train_aug = np.array([augment_keypoints(x) for x in X_train])
X_train_combined = np.concatenate([X_train, X_train_aug])
y_train_combined = np.concatenate([y_train, y_train])
cobb_train_combined = np.concatenate([cobb_train, cobb_train])

# Compute class weights
class_weights = compute_class_weight('balanced', classes=np.unique(y_train), y=y_train)
class_weights_dict = {i: class_weights[i] for i in range(len(class_weights))}

# Prepare sample weights
sample_weights_combined = {
    'classification': np.array([class_weights_dict[label] for label in y_train_combined]),
    'cobb_angle': np.ones(len(y_train_combined))
}

# Build ResNet-style model
def residual_block(x, units):
    shortcut = x
    x = layers.Dense(units, activation='relu', kernel_regularizer=l2(0.001))(x)
    x = layers.BatchNormalization(momentum=0.9)(x)
    x = layers.Dropout(0.2)(x)
    x = layers.Dense(units, activation='relu', kernel_regularizer=l2(0.001))(x)
    x = layers.BatchNormalization(momentum=0.9)(x)
    if shortcut.shape[-1] != units:
        shortcut = layers.Dense(units, kernel_regularizer=l2(0.001))(shortcut)
    x = layers.Add()([shortcut, x])
    return x

def build_resnet_model(input_shape):
    inputs = keras.Input(shape=input_shape)
    x = layers.Dense(128, activation='relu', kernel_regularizer=l2(0.001))(inputs)
    x = layers.BatchNormalization(momentum=0.9)(x)
    for units in [256, 512, 1024]:
        x = residual_block(x, units)
        x = layers.Dropout(0.2)(x)
    x = layers.Dense(512, activation='relu', kernel_regularizer=l2(0.001))(x)
    class_output = layers.Dense(4, activation='softmax', name='classification')(x)
    angle_output = layers.Dense(1, activation='linear', name='cobb_angle')(x)

    model = keras.Model(inputs=inputs, outputs=[class_output, angle_output])
    model.compile(
        optimizer=keras.optimizers.Adam(learning_rate=0.0001),
        loss={'classification': 'sparse_categorical_crossentropy', 'cobb_angle': 'mse'},
        metrics={'classification': 'accuracy', 'cobb_angle': 'mae'}
    )
    return model

In [ ]:
# Build and compile the model (ensure this matches your definition)
def build_resnet_model(input_shape):
    inputs = keras.Input(shape=input_shape)
    x = layers.Dense(128, activation='relu', kernel_regularizer=l2(0.001))(inputs)
    x = layers.BatchNormalization(momentum=0.9)(x)
    for units in [256, 512, 1024]:
        x = residual_block(x, units)
        x = layers.Dropout(0.2)(x)
    x = layers.Dense(512, activation='relu', kernel_regularizer=l2(0.001))(x)
    class_output = layers.Dense(4, activation='softmax', name='classification')(x)
    angle_output = layers.Dense(1, activation='linear', name='cobb_angle')(x)

    model = keras.Model(inputs=inputs, outputs=[class_output, angle_output])
    model.compile(
        optimizer=keras.optimizers.Adam(learning_rate=0.0001),
        loss={'classification': 'sparse_categorical_crossentropy', 'cobb_angle': 'mse'},
        metrics={'classification': 'accuracy', 'cobb_angle': 'mae'}
    )
    return model

# Create the model
model = build_resnet_model(input_shape=(X.shape[1],))

# Debug shapes and output names
print("Model output names:", model.output_names)
print("X_train_combined shape:", X_train_combined.shape)
print("y_train_combined shape:", y_train_combined.shape)
print("cobb_train_combined shape:", cobb_train_combined.shape)
print("Sample weights keys:", sample_weights_combined.keys())

# Ensure sample weights match the number of samples
assert len(sample_weights_combined['classification']) == X_train_combined.shape[0], "Sample weights mismatch for classification"
assert len(sample_weights_combined['cobb_angle']) == X_train_combined.shape[0], "Sample weights mismatch for cobb_angle"

# Train the model
callbacks = [
    keras.callbacks.EarlyStopping(monitor='val_loss', patience=20, restore_best_weights=True),
    keras.callbacks.ReduceLROnPlateau(monitor='val_loss', factor=0.5, patience=10, min_lr=1e-6)
]

print("Sample weights classification shape:", sample_weights_combined['classification'].shape)
print("Sample weights cobb_angle shape:", sample_weights_combined['cobb_angle'].shape)
print("Sample weights classification type:", type(sample_weights_combined['classification']))
print("Sample weights cobb_angle type:", type(sample_weights_combined['cobb_angle']))

# Convert sample weights to list format (order matches model.output_names)
sample_weights_list = [
    sample_weights_combined['classification'],
    sample_weights_combined['cobb_angle']
]

# Train with sample weights as a list
callbacks = [
    keras.callbacks.EarlyStopping(monitor='val_loss', patience=20, restore_best_weights=True),
    keras.callbacks.ReduceLROnPlateau(monitor='val_loss', factor=0.5, patience=10, min_lr=1e-6)
]

history = model.fit(
    X_train_combined,
    [y_train_combined, cobb_train_combined],  # Targets as list
    epochs=200,
    batch_size=256,
    validation_data=(X_val, [y_val, cobb_val]),  # Validation as list
    sample_weight=sample_weights_list,  # Sample weights as list
    callbacks=callbacks,
    verbose=1
)

# Evaluate model
eval_results = model.evaluate(X_test, {'classification': y_test, 'cobb_angle': cobb_test})
print(f'Test Loss: {eval_results[0]}, Classification Accuracy: {eval_results[1]}, Cobb Angle MAE: {eval_results[2]}')

# Confusion Matrix
y_pred = np.argmax(model.predict(X_test)[0], axis=1)
conf_matrix = confusion_matrix(y_test, y_pred)
sns.heatmap(conf_matrix, annot=True, cmap='Greens', fmt='d',
            xticklabels=['Normal', 'Mild', 'Moderate', 'Severe'],
            yticklabels=['Normal', 'Mild', 'Moderate', 'Severe'])
plt.xlabel('Predicted')
plt.ylabel('Actual')
plt.title('Confusion Matrix')
plt.show()

# Classification Report
print("\033[92mClassification Report:\033[0m")
print(classification_report(y_test, y_pred, target_names=['Normal', 'Mild', 'Moderate', 'Severe']))

# Plot training loss and accuracy
plt.figure(figsize=(12, 5))
plt.subplot(1, 2, 1)
plt.plot(history.history['classification_loss'], label='Train Loss', color='hotpink')
plt.plot(history.history['val_classification_loss'], label='Validation Loss', color='limegreen')
plt.xlabel('Epochs')
plt.ylabel('Loss')
plt.legend()
plt.title('Model Training Loss')
plt.grid(True)

plt.subplot(1, 2, 2)
plt.plot(history.history['classification_accuracy'], label='Train Accuracy', color='hotpink')
plt.plot(history.history['val_classification_accuracy'], label='Validation Accuracy', color='limegreen')
plt.xlabel('Epochs')
plt.ylabel('Accuracy')
plt.legend()
plt.title('Model Training Accuracy')
plt.grid(True)
plt.show()

# Plot Cobb angle predictions
cobb_pred = model.predict(X_test)[1].flatten() * 90  # Denormalize
plt.scatter(cobb_test * 90, cobb_pred, c='hotpink')
plt.plot([0, 90], [0, 90], 'limegreen', ls='--')
plt.xlabel('Actual Cobb Angle')
plt.ylabel('Predicted Cobb Angle')
plt.title('Cobb Angle Prediction')
plt.grid(True)
plt.show()

# Save model
model.save("/content/drive/MyDrive/scoliosis_optimized_resnet.h5")
print("Model saved to Google Drive as 'scoliosis_optimized_resnet.h5'")

Model output names: ListWrapper(['classification', 'cobb_angle'])
X_train_combined shape: (16756, 34)
y_train_combined shape: (16756,)
cobb_train_combined shape: (16756,)
Sample weights keys: dict_keys(['classification', 'cobb_angle'])
Sample weights classification shape: (16756,)
Sample weights cobb_angle shape: (16756,)
Sample weights classification type: <class 'numpy.ndarray'>
Sample weights cobb_angle type: <class 'numpy.ndarray'>
Epoch 1/200
66/66 ━━━━━━━━━━━━━━━━━━━━ 23s 185ms/step - classification_accuracy: 0.4575 - classification_loss: 1.4866 - cobb_angle_loss: 2.0338 - cobb_angle_mae: 1.0811 - loss: 8.4337 - val_classification_accuracy: 0.6183 - val_classification_loss: 0.6973 - val_cobb_angle_loss: 0.2702 - val_cobb_angle_mae: 0.4381 - val_loss: 5.9906 - learning_rate: 1.0000e-04
Epoch 2/200
66/66 ━━━━━━━━━━━━━━━━━━━━ 19s 160ms/step - classification_accuracy: 0.5990 - classification_loss: 0.9589 - cobb_angle_loss: 0.8398 - cobb_angle_mae: 0.7093 - loss: 6.6919 - val_classifi